In [1]:
%run C:/Users/MohammedSB/Desktop/projects/Hypertension/Requirements.ipynb

In [2]:
# Seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
PATH = r"D:\\MohammedSB\\Fundus\\"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = r"C:\Users\MohammedSB\Desktop\projects\Hypertension\models\\"

In [5]:
train_set, test_set = get_datasets(val=False)

In [6]:
train_transform = T.Compose([
    T.Resize((500, 500)),
    T.ToTensor(),
    T.RandomHorizontalFlip(0.5),
    T.RandomRotation(degrees=(0, 360)),
    T.GaussianBlur(5),
    T.Normalize(mean=[0.3675, 0.1760, 0.1047], std=[0.2289, 0.1180, 0.0748]),
])

test_transform = T.Compose([
    T.Resize((500, 500)),
    T.ToTensor(),
    T.Normalize(mean=[0.3675, 0.1760, 0.1047], std=[0.2289, 0.1180, 0.0748]),
])

train_dataset = HypertensionDataset(CSV_PATH, train_set, train_transform=train_transform)
test_dataset = HypertensionDataset(CSV_PATH, test_set, test_transform=test_transform)

train_loader, test_loader = train_test_loader(train_dataset, test_dataset, batch_size=8)

# val_dataset = HypertensionDataset(CSV_PATH, val_set, test_transform=test_transform)
# val_loader = DataLoader(val_dataset, batch_size=8)

In [7]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=64),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=64)

fusion_model = nn.Sequential(
    nn.Linear(in_features=128, out_features=256),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=512),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=256, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=8),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=8, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

In [8]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

early_stop = {
    "patience": 5,
    "min_delta": 0.001,
    "multip": 5,
}

optimizer = torch.optim.Adam([
    {'params': image_model.parameters(), 'lr': 1e-4},
    {'params': tabular_model.parameters(), 'lr': 1e-4},
    {'params': fusion_model.parameters(), 'lr': 1e-4}],
)

criterion = nn.BCEWithLogitsLoss()

In [9]:
epochs = 200
metrics = train_val(epochs=epochs, models=models, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
                    val_loader=test_loader, device=device, early_stop=early_stop)

Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

Beginning Training: 

Epoch 1/200


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.697415,    Accuracy: 51.48%,    Correct Counter: 471/915,    F1 Score: 0.00,    Precision: 1.00,    Recall:  0.00 



Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

...

Validation Set:
    Average Loss:  0.598120,    Accuracy: 65.35%,    Correct Counter: 149/228,    F1 Score: 0.71,    Precision: 0.60,    Recall:  0.87 

Epoch 20/200


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.585161,    Accuracy: 64.37%,    Correct Counter: 589/915,    F1 Score: 0.62,    Precision: 0.64,    Recall:  0.60 



Batch in Progress: 100%|████████████████████████████████████████████| 29/29 [00:07<00:00,  3.97it/s]

Validation Set:
    Average Loss:  0.602931,    Accuracy: 65.35%,    Correct Counter: 149/228,    F1 Score: 0.71,    Precision: 0.61,    Recall:  0.86 



In [11]:
torch.save(image_model, MODELS_PATH + "\ImageModel.pth")
torch.save(tabular_model, MODELS_PATH + "\TabularModel.pth")
torch.save(fusion_model, MODELS_PATH + "\FusionModel.pth")